# Scraping Concerts - Lab

## Introduction

Now that you've seen how to scrape a simple website, it's time to again practice those skills on a full-fledged site!
In this lab, you'll practice your scraping skills on a music website: https://www.residentadvisor.net.
## Objectives

You will be able to:
* Create a full scraping pipeline that involves traversing over many pages of a website, dealing with errors and storing data

## View the Website

For this lab, you'll be scraping the https://www.residentadvisor.net website. Start by navigating to the events page [here](https://www.residentadvisor.net/events) in your browser.

<img src="images/ra.png">

In [ ]:
# Load the https://www.residentadvisor.net/events page in your browser.

## Open the Inspect Element Feature

Next, open the inspect element feature from your web browser in order to preview the underlying HTML associated with the page.

In [ ]:
# Open the inspect element feature in your browser

## Write a Function to Scrape all of the Events on the Given Page Events Page

The function should return a Pandas DataFrame with columns for the Event_Name, Venue, Event_Date and Number_of_Attendees.

In [1]:
import re
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [3]:
response = requests.get("https://www.residentadvisor.net/events/us/newyork")
soup = BeautifulSoup(response.content, 'html.parser')

In [4]:
event_listings = soup.find('div', id="event-listing")

entries = event_listings.findAll('li')
print(len(entries), entries[0])

133 <li><p class="eventDate date"><a href="/events.aspx?ai=8&amp;v=day&amp;mn=11&amp;yr=2019&amp;dy=6"><span>Wed, 06 Nov 2019 /</span></a></p></li>


In [5]:
rows = []
for entry in entries:
    date = entry.find('p', class_="eventDate date")
    event = entry.find('h1', class_="event-title")
    if event:
        details = event.text.split(' at ')
        event_name = details[0].strip()
        venue = details[1].strip()
        try:
            n_attendees = int(re.match("(\d*)", entry.find('p', class_="attending").text)[0])
        except:
            n_attendees = np.nan
        rows.append([event_name, venue, cur_date, n_attendees])
    elif date:
        cur_date = date.text
    else:
        continue
df = pd.DataFrame(rows)
df.head()

,0,1,2,3
0,Personal Space / Feel Free NYC's 1-Year Party ...,Jupiter Disco,"Wed, 06 Nov 2019 /",13.0
1,"DJ Monchan, Shawn Dub, Disco_bobulated, For Fu...",Good Room,"Wed, 06 Nov 2019 /",8.0
2,Ūndisclosed - Scorpios Celebration,TBA Brooklyn,"Wed, 06 Nov 2019 /",6.0
3,Mesmarize 01,444 Club,"Wed, 06 Nov 2019 /",6.0
4,Twice as Proper - Keybar with Feral (Hypnus Re...,Keybar,"Wed, 06 Nov 2019 /",NaN


In [8]:
def scrape_events(events_page_url):
    response = requests.get(events_page_url)
    soup = BeautifulSoup(response.content, 'html.parser')
    entries = event_listings.findAll('li')
    rows = []
    for entry in entries:
        date = entry.find('p', class_="eventDate date")
        event = entry.find('h1', class_="event-title")
        if event:
            details = event.text.split(' at ')
            event_name = details[0].strip()
            venue = details[1].strip()
            try:
                n_attendees = int(re.match("(\d*)", entry.find('p', class_="attending").text)[0])
            except:
                n_attendees = np.nan
            rows.append([event_name, venue, cur_date, n_attendees])
        elif date:
            cur_date = date.text
        else:
            continue
    df = pd.DataFrame(rows)
    df.head()
    df.columns = ["Event_Name", "Venue", "Event_Date", "Number_of_Attendees"]
    return df

## Write a Function to Retrieve the URL for the Next Page

In [9]:
soup.find('a', attrs={'ga-event-action':"Next "}).attrs['href']

'/events/us/newyork/week/2019-11-13'

In [10]:
def next_page(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    url_ext = soup.find('a', attrs={'ga-event-action':"Next "}).attrs['href']
    next_page_url = "https://www.residentadvisor.net" + url_ext
    return next_page_url

## Scrape the Next 1000 Events for Your Area

Display the data sorted by the number of attendees. If there is a tie for the number attending, sort by event date.

In [18]:
dfs = []
total_rows = 0
cur_url = "https://www.residentadvisor.net/events/us/newyork"
while total_rows <= 1000:
    df = scrape_events(cur_url)
    dfs.append(df)
    total_rows += len(df)
    cur_url = next_page(cur_url)
df = pd.concat(dfs)
df = df.iloc[:1000]
df.sort_values(['Number_of_Attendees', 'Event_Date'],ascending=[False, False])
print(len(df))
df.head()


1000


,Event_Name,Venue,Event_Date,Number_of_Attendees
0,Personal Space / Feel Free NYC's 1-Year Party ...,Jupiter Disco,"Wed, 06 Nov 2019 /",13.0
1,"DJ Monchan, Shawn Dub, Disco_bobulated, For Fu...",Good Room,"Wed, 06 Nov 2019 /",8.0
2,Ūndisclosed - Scorpios Celebration,TBA Brooklyn,"Wed, 06 Nov 2019 /",6.0
3,Mesmarize 01,444 Club,"Wed, 06 Nov 2019 /",6.0
4,Twice as Proper - Keybar with Feral (Hypnus Re...,Keybar,"Wed, 06 Nov 2019 /",NaN


## Summary 

Congratulations! In this lab, you successfully developed a pipeline to scrape a website for concert event information!